In [62]:
import pandas as pd
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import classification_report
import sklearn
import HTMLParser
import re
from nltk.stem.porter import PorterStemmer as porterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [54]:
def cleanTweet(originalTweet):
    htmlParser = HTMLParser.HTMLParser()

    tweet = originalTweet
    # convert to lower case
    tweet = tweet.lower()
    #tweet = htmlParser.unescape(originalTweet);
    #tweet = tweet.decode('windows-1252').encode('ascii', 'ignore')
    #tweet = tweet.decode('windows-1252')
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #tweet = re.sub(r'[^\x00-\xFF]+', r'', tweet)
    #tweet = re.sub(r'[^\x00-\x7F]+', r'', tweet)
    #tweet = tweet.decode('utf-8').strip()
    #tweet = tweet.decode('unicode_escape').encode('ascii','ignore')
    #tweet = tweet.encode('ascii','ignore')
    tweet = ''.join([i if ord(i) < 128 else ' ' for i in tweet])
    
    # remove URLs in tweet
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)

    # remove strings starting with @ in tweet
    tweet = re.sub(r'(\s)@\w+', r'', tweet)
    tweet = re.sub(r'@\w+', r'', tweet)
    
    # remove HTML tags from tweet
    tweet = re.sub('<[^<]+?>', '', tweet)

    # separates words joined with capital words.
    # E.g. DisplayIsAweson to Display Is Awesom
    #tweet = " ".join(re.findall('[A-Z][^A-Z]*', tweet));

    # remove exclamation and other unnecessary characters
    tweet = re.sub(r'[<>!#@$:.,%\?-]+', r'', tweet)

    
    
    tweet = re.sub(r' a ', r' ', tweet)
    tweet = re.sub(r' with ', r' ', tweet)
    tweet = re.sub(r' is ', r' ', tweet)
    tweet = re.sub(r' to ', r' ', tweet)
    tweet = re.sub(r' the ', r' ', tweet)
    tweet = re.sub(r' and ', r' ', tweet)
    tweet = re.sub(r' of ', r' ', tweet)
    tweet = re.sub(r' in ', r' ', tweet)
    tweet = re.sub(r' for ', r' ', tweet)
    tweet = re.sub(r' all ', r' ', tweet)
    tweet = re.sub(r' it ', r' ', tweet)
    tweet = re.sub(r' that ', r' ', tweet)
    tweet = re.sub(r' for ', r' ', tweet)
    tweet = re.sub(r' be ', ' ', tweet)
    tweet = re.sub(r' your ', r' ', tweet)
    tweet = re.sub(r' our ', r' ', tweet)
    tweet = re.sub(r' are ', r' ', tweet)
    tweet = re.sub(r' by ', r' ', tweet)
    tweet = re.sub(r' this ', r' ', tweet)
    tweet = re.sub(r' thi ', r' ', tweet)
    tweet = re.sub(r' from ', r' ', tweet)
    tweet = re.sub(r' or ', r' ', tweet)
    tweet = re.sub(r' website_redact ', r' ', tweet)
    tweet = re.sub(r' at ', r' ', tweet)
    tweet = re.sub(r' on ', r' ', tweet)
    tweet = re.sub(r' href= ', r' ', tweet)
    tweet = re.sub(r'\btarget=_blank\b', r' ', tweet)
    tweet = re.sub(r' also ', r' ', tweet)
    tweet = re.sub(r' can ', r' ', tweet)
    tweet = re.sub(r' call/text ', r' ', tweet)
    tweet = re.sub(r' email ', r' ', tweet)
    tweet = re.sub(r' viewingcontact ', r' ', tweet)
    tweet = re.sub(r' id ', r' ', tweet)
    tweet = re.sub(r' veri ', r' ', tweet)
    tweet = re.sub(r' kagglemanagercom ', r' ', tweet)
    tweet = re.sub(r' website_redact', r' ', tweet)
    # remove extra white spaces
    tweet = re.sub(r'\s+', r' ', tweet)
    
    # stemming
    stemmer = porterStemmer()
    stemmedTweet = [stemmer.stem(word) for word in tweet.split(" ")]
    stemmedTweet = " ".join(stemmedTweet)
    
    tweet = str(stemmedTweet)
    tweet = tweet.replace("'", "")
    
    return tweet

In [3]:
df = pd.read_json('train.json')

In [4]:
df['interest_level'] = df['interest_level'].astype('category')
df['interest_level_codes'] = df['interest_level'].cat.codes

In [5]:
#df[['interest_level','interest_level_codes']].head(2)
#print(df['interest_level_codes'].hist())
#df.to_csv('generated-data-with-codes.csv',sep=',', encoding='utf-8')
df['interest_level_codes'].value_counts()

1    34284
2    11229
0     3839
Name: interest_level_codes, dtype: int64

In [6]:
df.head(2)

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level_codes
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,2
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,1


In [7]:
y_label = df['interest_level_codes']
feature_matrix_df = df[['bathrooms','bathrooms','price']];

In [8]:
feature_matrix_df.head(2)

,bathrooms,bathrooms,price
10,1.5,1.5,3000
10000,1.0,1.0,5465


In [9]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix_df,y_label, test_size=0.25)
print len(X_train)
print len(X_test)
print len(y_train)
print len(y_test)

37014
12338
37014
12338


In [10]:
multinomialNBClassifier = MultinomialNB();

In [11]:
multinomialNBClassifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
predicted_labels = multinomialNBClassifier.predict(X_test)

In [13]:
actual_class = y_test.tolist()
predicted_class = predicted_labels.tolist()

In [14]:
target_names = ['0', '1', '2']

In [15]:
print(classification_report(actual_class, predicted_class, target_names=target_names))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       974
          1       0.69      1.00      0.82      8566
          2       0.00      0.00      0.00      2798

avg / total       0.48      0.69      0.57     12338



In [16]:
predicted_prob_labels = multinomialNBClassifier.predict_proba(X_test);

In [17]:
#print len(predicted_prob_labels)
prob_labels_list = predicted_prob_labels.tolist()
print prob_labels_list

[[0.07084188625894171, 0.706354553739186, 0.2228035600018715], [0.07497201446770832, 0.695440220698575, 0.22958776483371504], [0.09294231470632668, 0.6505436569917286, 0.2565140283019434], [0.06768613898082754, 0.7148643436242925, 0.2174495173948815], [0.06202762065728184, 0.7305213184763582, 0.2074510608663584], [0.08768658238690526, 0.6571496856181582, 0.25516373199493403], [0.0756909402016323, 0.6935651046916704, 0.23074395510669596], [0.0512673252769972, 0.7570937601020284, 0.1916389146209777], [0.08241283435634311, 0.6763672620838934, 0.24121990355976425], [0.06420410780705836, 0.7244360829218389, 0.21135980927110462], [0.08241283435634311, 0.6763672620838934, 0.24121990355976425], [0.07862220936888843, 0.6859925823964665, 0.23538520823464623], [0.07944401144240391, 0.6838901089546396, 0.23666587960295757], [0.0739039593419559, 0.6982392604049367, 0.22785678025310588], [0.06808245972171438, 0.7137872042826815, 0.21813033599560355], [0.10882619381010719, 0.6071806469326712, 0.28399

In [18]:
description_df = df[['description','interest_level_codes']]
description_df.head(5)
description_df.to_csv('description.csv',sep=',', encoding='utf-8')


In [19]:
description_column = description_df['description']
label_column = description_df['interest_level_codes']

In [20]:
description_col_list = description_column.tolist()
label_col_list = label_column.tolist()

In [57]:
# clean all the tweets
i = 0;
cleanedDescriptionList = []
for description in description_col_list:
    #tweet.encode('utf-8').strip()
    #tweet = tweet.decode("utf8").encode('ascii', 'ignore')
    #print (i ,),
    cleanedDescription = cleanTweet(description).encode('ascii', 'ignore').strip();
    cleanedDescriptionList.append(cleanedDescription);
    
    i += 1

In [60]:
trainDescription, testDescription, trainLabels, testLabels = train_test_split(cleanedDescriptionList, label_col_list, 
                                                                              test_size=0.20, random_state = 40);

In [64]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,5),lowercase=True)

In [66]:
train_vectors = vectorizer.fit_transform(trainDescription);
test_vectors = vectorizer.transform(testDescription);
total_vectors = vectorizer.fit_transform(cleanedDescriptionList);


In [68]:
multinomialNBClassifier = MultinomialNB();
multinomialNBClassifier.fit(train_vectors, trainLabels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [72]:
predicted_labels = multinomialNBClassifier.predict(test_vectors)
predicted_class = predicted_labels.tolist()

In [74]:
target_names = ['0', '1', '2']

In [76]:
print(classification_report(testLabels, predicted_class, target_names=target_names))

             precision    recall  f1-score   support

          0       0.25      0.07      0.12       738
          1       0.73      0.93      0.82      6911
          2       0.42      0.17      0.24      2222

avg / total       0.63      0.69      0.64      9871

